In [ ]:
!pip install -q transformers

In [ ]:
!pip install -q faiss-cpu

In [ ]:
!wget https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-nq-adv-hn-train.json.gz
!gunzip biencoder-nq-adv-hn-train.json.gz

In [1]:
import json
import random
from dataclasses import dataclass
import numpy as np
from transformers import AutoTokenizer
import os
import uuid
from transformers import AutoConfig, AutoTokenizer, TFAutoModel
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tqdm import tqdm_notebook

In [3]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
print(resolver.master())
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)
strategy

grpc://10.0.0.2:8470


# Data Preparation

In [4]:
def read_dpr_json(file, max_samples=None, num_hard_negatives=1, num_positives=1, shuffle_negatives=True, shuffle_positives=False):
    
    dicts = json.load(open(file, encoding='utf-8'))

    #if max_samples:
    #    dicts = random.sample(dicts, min(max_samples, len(dicts)))

    # convert DPR dictionary to standard dictionary
    query_json_keys = ["question", "questions", "query"]
    positive_context_json_keys = ["positive_contexts", "positive_ctxs", "positive_context", "positive_ctx"]
    hard_negative_json_keys = ["hard_negative_contexts", "hard_negative_ctxs", "hard_negative_context", "hard_negative_ctx"]
    standard_dicts = []
    for i in tqdm_notebook(range(len(dicts))):
        dict = dicts[i]
        sample = {}
        positive_passages = []
        negative_passages = []
        for key, val in dict.items():
            if key in query_json_keys:
                sample["query"] = val
            elif key in positive_context_json_keys:
                if shuffle_positives:
                    random.shuffle(val)
                for passage in val[:num_positives]:
                    positive_passages.append({
                        "title": passage["title"],
                        "text": passage["text"],
                        "label": "positive"})
            elif key in hard_negative_json_keys:
                if shuffle_negatives:
                    random.shuffle(val)
                for passage in val[:num_hard_negatives]:
                    negative_passages.append({
                        "title": passage["title"],
                        "text": passage["text"],
                        "label": "hard_negative"})
        sample["passages"] = positive_passages + negative_passages
        if len(sample["passages"]) == num_positives+num_hard_negatives:
            standard_dicts.append(sample)
        if max_samples:
            if len(standard_dicts)==max_samples:
                break
    return standard_dicts

In [5]:
dicts = read_dpr_json("biencoder-nq-adv-hn-train.json",max_samples=60000, num_hard_negatives=3)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


  0%|          | 0/69639 [00:00<?, ?it/s]

In [6]:
@dataclass
class DataConfig:
    num_positives = 1
    num_hard_negatives = 3

data_config = DataConfig()

class ModelConfig:
    passage_max_seq_len = 156
    query_max_seq_len = 64
    batch_size_per_replica = 128
    epochs = 40
    learning_rate = 2e-5
    num_warmup_steps = 1234
    dropout = 0.1
    model_name = "google/bert_uncased_L-4_H-512_A-8"
    
model_config = ModelConfig()

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_config.model_name)

In [8]:
def encode_query_passage(tokenizer, dicts, model_config, data_config):
    passage_input_ids = []
    passage_token_type_ids = []
    passage_attention_mask = []
    
    queries = []
    for i in tqdm_notebook(range(len(dicts))):
        di = dicts[i]
        di_query = di['query']
        di_passages = di['passages']
        di_positives = [(pi['title'],pi['text']) for pi in di_passages if pi['label']=="positive"]
        di_negatives = [(ni['title'],ni['text']) for ni in di_passages if ni['label']=="hard_negative"]
        
        if data_config.num_positives == len(di_positives) and data_config.num_hard_negatives == len(di_negatives):
            
            queries.append(di_query)
            i_passages = di_positives + di_negatives
            i_passage_inputs = tokenizer.batch_encode_plus(
                        i_passages,
                        max_length=model_config.passage_max_seq_len,
                        add_special_tokens=True,
                        truncation=True,
                        truncation_strategy='longest_first',
                        padding="max_length",
                        return_token_type_ids=True,
                    )
            passage_input_ids.append(np.array(i_passage_inputs['input_ids']))
            passage_token_type_ids.append(np.array(i_passage_inputs['token_type_ids']))
            passage_attention_mask.append(np.array(i_passage_inputs['attention_mask']))
    
    print("len queries:", len(queries))
    query_inputs = tokenizer.batch_encode_plus(
                    queries,
                    max_length=model_config.query_max_seq_len,
                    add_special_tokens=True,
                    truncation=True,
                    truncation_strategy='longest_first',
                    padding="max_length",
                    return_token_type_ids=True,
                    return_tensors="np"
                    )
    
    
    return {
        "query_input_ids": query_inputs['input_ids'],
        "query_token_type_ids": query_inputs['token_type_ids'],
        "query_attention_mask": query_inputs['attention_mask'],
        "passage_input_ids": np.array(passage_input_ids),
        "passage_token_type_ids": np.array(passage_token_type_ids),
        "passage_attention_mask": np.array(passage_attention_mask)
    } 

In [9]:
X = encode_query_passage(tokenizer, dicts, model_config, data_config)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/60000 [00:00<?, ?it/s]

len queries: 60000


# Modeling

In [10]:
from transformers import create_optimizer

In [11]:
class QueryModel(tf.keras.Model):
    def __init__(self, model_config, **kwargs):
        super().__init__(**kwargs)
        self.query_encoder = TFAutoModel.from_pretrained(model_config.model_name, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(model_config.dropout)
        
    def call(self, inputs, training=False, **kwargs):
        
        pooled_output = self.query_encoder(inputs, training=training, **kwargs)[1]
        pooled_output = self.dropout(pooled_output, training=training)
        print("Q pooled_output :",pooled_output.shape)
        return pooled_output

In [12]:
class PassageModel(tf.keras.Model):
    def __init__(self, model_config, **kwargs):
        super().__init__(**kwargs)
        self.passage_encoder = TFAutoModel.from_pretrained(model_config.model_name, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(model_config.dropout)
        
    def call(self, inputs, training=False, **kwargs):
   
        pooled_output = self.passage_encoder(inputs, training=training, **kwargs)[1]
        pooled_output = self.dropout(pooled_output, training=training)
        print("P pooled_output :", pooled_output.shape)
        return pooled_output

In [13]:
def cross_replica_concat(values, v_shape):

    context = tf.distribute.get_replica_context()
    gathered = context.all_gather(values, axis=0)

    return tf.roll(
      gathered,
      -context.replica_id_in_sync_group * values.shape[0], #v_shape,#values.shape[0],
      axis=0
    )

class BiEncoderModel(tf.keras.Model):
    def __init__(self, query_encoder, passage_encoder, num_passages_per_question, model_config, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.query_encoder = query_encoder
        self.passage_encoder = passage_encoder
        self.num_passages_per_question = num_passages_per_question
        self.model_config = model_config
        
        self.loss_tracker = tf.keras.metrics.Mean(name="loss")
        self.loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE, from_logits=True)
        
    def calculate_loss(self, logits):
        
        num_queries = tf.shape(logits)[0]
        num_candidates = tf.shape(logits)[1]
        
        labels = tf.convert_to_tensor([i for i in range(0,(GLOBAL_BATCH_SIZE*self.num_passages_per_question),self.num_passages_per_question)])
        
        loss = self.loss_fn(labels, logits)
        scale_loss = tf.reduce_sum(loss) * (1. / GLOBAL_BATCH_SIZE)
        return scale_loss
    
    def passage_forward(self, X):
        input_shape = (self.model_config.batch_size_per_replica * self.num_passages_per_question, self.model_config.passage_max_seq_len)
        input_ids = tf.reshape(X["passage_input_ids"], input_shape)
        attention_mask = tf.reshape(X["passage_attention_mask"], input_shape)
        token_type_ids = tf.reshape(X["passage_token_type_ids"], input_shape)
        outputs = self.passage_encoder([input_ids, attention_mask, token_type_ids], training=True)
        return outputs


    def query_forward(self, X):
        input_shape = (self.model_config.batch_size_per_replica, self.model_config.query_max_seq_len)
        input_ids = tf.reshape(X["query_input_ids"], input_shape)
        attention_mask = tf.reshape(X["query_attention_mask"], input_shape)
        token_type_ids = tf.reshape(X["query_token_type_ids"], input_shape)
        outputs = self.query_encoder([input_ids, attention_mask, token_type_ids], training=True)
        return outputs
    
    def train_step(self, X):
        
        with tf.GradientTape() as tape:
            passage_embeddings = self.passage_forward(X)
            query_embeddings = self.query_forward(X)
            
            global_passage_embeddings = cross_replica_concat(passage_embeddings, 32)
            global_query_embeddings = cross_replica_concat(query_embeddings, 16)
            
            print("Global Passage Shape :", global_passage_embeddings.shape)
            print("Global Query Shape :", global_query_embeddings.shape)
            
            similarity_scores = tf.linalg.matmul(global_query_embeddings, global_passage_embeddings, transpose_b=True)
            
            loss = self.calculate_loss(similarity_scores)
            loss = loss / strategy.num_replicas_in_sync
            
        # Backward pass
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # Monitor loss
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}
    
    @property
    def metrics(self):
        return [self.loss_tracker]

In [14]:
BATCH_SIZE_PER_REPLICA = model_config.batch_size_per_replica
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
N_EPOCHS = model_config.epochs
one_epoch_steps = int(len(dicts)/GLOBAL_BATCH_SIZE)

In [15]:
one_epoch_steps, N_EPOCHS, BATCH_SIZE_PER_REPLICA, GLOBAL_BATCH_SIZE

(58, 40, 128, 1024)

In [16]:
with strategy.scope():
    query_encoder = QueryModel(model_config)
    passage_encoder = PassageModel(model_config)
    bi_model = BiEncoderModel(query_encoder, passage_encoder, num_passages_per_question=data_config.num_positives+data_config.num_hard_negatives, model_config=model_config)
    optimizer, lr_schedule = create_optimizer(init_lr=model_config.learning_rate, num_train_steps=N_EPOCHS*one_epoch_steps, num_warmup_steps=(N_EPOCHS*one_epoch_steps)*0.1)
    bi_model.compile(optimizer=optimizer)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

In [17]:
with strategy.scope():
    train_ds = tf.data.Dataset.from_tensor_slices(X).shuffle(GLOBAL_BATCH_SIZE * 10).prefetch(buffer_size=tf.data.experimental.AUTOTUNE).batch(GLOBAL_BATCH_SIZE, drop_remainder=True)

In [18]:
bi_model.fit(train_ds, epochs=N_EPOCHS)

Epoch 1/40
P pooled_output : (512, 512)
Q pooled_output : (128, 512)
Global Passage Shape : (4096, 512)
Global Query Shape : (1024, 512)
P pooled_output : (512, 512)
Q pooled_output : (128, 512)
Global Passage Shape : (4096, 512)
Global Query Shape : (1024, 512)
58/58 [==============================] - 85s 451ms/step - loss: 5.4331
Epoch 2/40
58/58 [==============================] - 26s 452ms/step - loss: 1.7092
Epoch 3/40
58/58 [==============================] - 26s 453ms/step - loss: 1.1920
Epoch 4/40
58/58 [==============================] - 26s 452ms/step - loss: 0.9585
Epoch 5/40
58/58 [==============================] - 26s 452ms/step - loss: 0.7851
Epoch 6/40
58/58 [==============================] - 26s 454ms/step - loss: 0.6400
Epoch 7/40
58/58 [==============================] - 26s 452ms/step - loss: 0.5393
Epoch 8/40
58/58 [==============================] - 26s 452ms/step - loss: 0.4755
Epoch 9/40
58/58 [==============================] - 26s 452ms/step - loss: 0.4338
Epoch 10/4

# Evaluation

In [19]:
!wget https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-nq-dev.json.gz
!gunzip biencoder-nq-dev.json.gz

--2021-06-26 17:25:34--  https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-nq-dev.json.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 256239282 (244M) [application/gzip]
Saving to: ‘biencoder-nq-dev.json.gz’

biencoder-nq-dev.js 100%[===================>] 244.37M  11.7MB/s    in 22s     

2021-06-26 17:25:58 (11.0 MB/s) - ‘biencoder-nq-dev.json.gz’ saved [256239282/256239282]



In [20]:
eval_dicts = read_dpr_json("biencoder-nq-dev.json", num_hard_negatives=30, shuffle_negatives=False)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


  0%|          | 0/6515 [00:00<?, ?it/s]

In [23]:
def combine_title_context(titles, texts):
    res = []
    for title, ctx in zip(titles, texts):
        if title is None:
            title = ""
        res.append(tuple((title, ctx)))
    return res

def process_single_example(passages):
    answer_index = -1
    titles = []
    texts = []
    for i in range(len(passages)):
        p = passages[i]
        titles.append(p['title'])
        texts.append(p['text'])
        if p['label']=="positive":
            answer_index = i

    res = combine_title_context(titles, texts)

    return res, answer_index

def process_examples(dicts):
    processed_passages = []
    queries = []
    answer_indexes = []
    global_answer_index = 0

    for i in range(len(dicts)):
        dict_ = dicts[i]
        query = dict_["query"]
        queries.append(query)

        passages = dict_["passages"]
        res, answer_index = process_single_example(passages)

        i_answer_index = global_answer_index + answer_index

        processed_passages.extend(res)
        answer_indexes.append(i_answer_index)

        global_answer_index = global_answer_index + len(passages)
    return queries, answer_indexes, processed_passages

In [22]:
queries, answer_indexes, processed_passages = process_examples(eval_dicts)
len(processed_passages), len(queries)

(199795, 6445)

In [24]:
def extracted_passage_embeddings(processed_passages, model_config):
    passage_inputs = tokenizer.batch_encode_plus(
                    processed_passages,
                    add_special_tokens=True,
                    truncation=True,
                    padding="max_length",
                    max_length=model_config.passage_max_seq_len,
                    return_token_type_ids=True
                )
    passage_embeddings = passage_encoder.predict([np.array(passage_inputs['input_ids']), 
                                                np.array(passage_inputs['attention_mask']), 
                                                np.array(passage_inputs['token_type_ids'])], 
                                                batch_size=512, 
                                                verbose=1)
    return passage_embeddings

In [25]:
passage_embeddings = extracted_passage_embeddings(processed_passages, model_config)

P pooled_output : (None, 512)
391/391 [==============================] - 22s 50ms/step


In [26]:
def extracted_query_embeddings(queries, model_config):
    query_inputs = tokenizer.batch_encode_plus(
                    queries,
                    add_special_tokens=True,
                    truncation=True,
                    padding="max_length",
                    max_length=model_config.query_max_seq_len,
                    return_token_type_ids=True
                )
    query_embeddings = query_encoder.predict([np.array(query_inputs['input_ids']), 
                                                np.array(query_inputs['attention_mask']), 
                                                np.array(query_inputs['token_type_ids'])], 
                                                batch_size=512, 
                                                verbose=1)
    return query_embeddings

In [27]:
query_embeddings = extracted_query_embeddings(queries, model_config)

Q pooled_output : (None, 512)
13/13 [==============================] - 8s 526ms/step


In [28]:
import faiss
faiss_index = faiss.IndexFlatL2(512)
faiss_index.add(passage_embeddings)

In [29]:
def get_k_accuracy(faiss_index, query_embeddings, answer_indexes, k):

    prob, index = faiss_index.search(query_embeddings, k=k)

    corrects = []
    for i in tqdm_notebook(range(len(answer_indexes))):
      i_index = index[i]
      i_count = len(np.where(i_index == answer_indexes[i])[0])
      if i_count >0:
            corrects.append((i, answer_indexes[i]))
    return corrects

In [30]:
top10_corrects = get_k_accuracy(faiss_index, query_embeddings, answer_indexes, k=10)
top20_corrects = get_k_accuracy(faiss_index, query_embeddings, answer_indexes, k=20)
top50_corrects = get_k_accuracy(faiss_index, query_embeddings, answer_indexes, k=50)
top100_corrects = get_k_accuracy(faiss_index, query_embeddings, answer_indexes, k=100)
top1000_corrects = get_k_accuracy(faiss_index, query_embeddings, answer_indexes, k=1000)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/6445 [00:00<?, ?it/s]

  0%|          | 0/6445 [00:00<?, ?it/s]

  0%|          | 0/6445 [00:00<?, ?it/s]

  0%|          | 0/6445 [00:00<?, ?it/s]

  0%|          | 0/6445 [00:00<?, ?it/s]

In [31]:
import pandas as pd
results = pd.DataFrame({'topK':[10,20,50,100,1000],
                       'total':[len(query_embeddings)]*5,
                       'correct_total':[len(top10_corrects), len(top20_corrects), len(top50_corrects), len(top100_corrects), len(top1000_corrects)]})

results['accuracy'] = (results['correct_total'] / results['total'])*100
results

,topK,total,correct_total,accuracy
0,10,6445,1153,17.889837
1,20,6445,1748,27.121800
2,50,6445,2631,40.822343
3,100,6445,3311,51.373157
4,1000,6445,5293,82.125679
